In [1]:
import pandas as pd
import datetime
import os
import numpy as np

In [2]:
week_list=['26','27','28','29','30','31']

In [3]:
Krogger_2017_128=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz Only - FY 2017 - FY 2018 YTD w.e. Aug 3 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)
Krogger_2017_64=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 64oz Only - FY 2017 - FY 2018 YTD w.e. Aug 11 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)

Krogger_2018=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz and 64oz FY 2018 YTD w.e. Sep 8 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)


In [4]:
def clean_df(df):
    df=df[['WEEK_NAME','UPC','SCANNED_RETAIL_DOLLARS','SCANNED_MOVEMENT','STO_NAME','STO_ZIP','STO_ADDRESS','ITM_SCN_PRC_GRP_DESC']]
    
    df['year']=df['WEEK_NAME'].apply(lambda x: x[:4])
    
    df['week']=np.where(df['year']=='2017',df['WEEK_NAME'].apply(lambda x: str(int(x[len(x)-3:len(x)-1])-1)),
                        df['WEEK_NAME'].apply(lambda x: x[len(x)-3:len(x)-1]))
    df['zip_cd']=df['STO_ZIP'].apply(lambda x: x[:5])
    df=df[df['week'].isin(week_list)]
    df=df.rename(columns={"SCANNED_RETAIL_DOLLARS":"sales","SCANNED_MOVEMENT":"units","STO_NAME":"store",
                          'STO_ADDRESS':'address',"ITM_SCN_PRC_GRP_DESC":"product"})
    df['sales']=df['sales'].astype(float)
    df['units']=df['units'].astype(float)
    return df[['store','year','week','zip_cd','address','sales','units','product']]

In [5]:
Krogger_2017_128=clean_df(Krogger_2017_128)
Krogger_2017_64=clean_df(Krogger_2017_64)
Krogger_2018=clean_df(Krogger_2018)
Krogger_2017_128=Krogger_2017_128[Krogger_2017_128['year']=='2017']
Krogger_2017_64=Krogger_2017_64[Krogger_2017_64['year']=='2017']
Krogger_2018=Krogger_2018[Krogger_2018['year']=='2018']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [6]:
Zip_DMA_mapping=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=2)
Zip_DMA_mapping=Zip_DMA_mapping.iloc[:,[0,2]]
Zip_DMA_mapping.columns=['zip_cd','DMA']
Zip_DMA_mapping=Zip_DMA_mapping.drop_duplicates('zip_cd')
Zip_DMA_mapping.shape

(40811, 2)

In [7]:
def func(x):
    y=len(list(set(x)))
    return y

Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
sales_available_stores=Krogger_both_year.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index().rename(columns={"year_week":"weeks_with_sales"})
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/Investigate_in_Krogger_RALEIGH_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
sales_available_stores.to_excel(writer,"summary_count_weeks",index=False)
writer.save()


# Combined Comparison

In [8]:
Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
Krogger_both_year=Krogger_both_year[Krogger_both_year['sales']>0]
Krogger_both_year.shape

(54584, 10)

In [9]:
len(Krogger_both_year['store_info'].unique())

2414

In [10]:
def func(x):
    y=len(list(set(x)))
    return y

In [11]:
Krogger_both_year_existing=Krogger_both_year.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing=Krogger_both_year_existing[Krogger_both_year_existing['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year=Krogger_both_year[Krogger_both_year['store_info'].isin(Krogger_both_year_existing)]



In [12]:
Krogger_both_year_compare=Krogger_both_year[Krogger_both_year['week'].isin(week_list)]
Krogger_both_year_compare=pd.merge(Krogger_both_year_compare,Zip_DMA_mapping,on='zip_cd',how="left")
Krogger_both_year_compare.shape

(52930, 11)

In [13]:
Krogger_both_year_compare[(Krogger_both_year_compare['year']=='2017')]['week'].unique()

array(['26', '27', '28', '29', '30', '31'], dtype=object)

In [14]:
DMA_15=['DALLAS-FT. WORTH','LOS ANGELES','ATLANTA','NEW YORK','CHICAGO','HOUSTON','PHOENIX (PRESCOTT)','CHARLOTTE',
            'WASHINGTON, DC (HAGRSTWN)','DENVER','PHILADELPHIA','GREENVLL-SPART-ASHEVLL-AND','BOSTON (MANCHESTER)',
            'DETROIT','MIAMI-FT. LAUDERDALE']

def In_18(x):
    if x in DMA_15:
        y="In_15_Mkts"
    else:
        y="Other"
    return y
Krogger_both_year_compare['15_Mkt_Ind']=Krogger_both_year_compare['DMA'].apply(lambda x: In_18(x))

In [15]:
df_YoY_compare_summary=Krogger_both_year_compare.groupby(['15_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()
df_YoY_compare_summary.to_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_DMA_YoY_"+str(datetime.datetime.now().date())+".csv",index=False)

In [16]:
df_YoY_compare_summary

,15_Mkt_Ind,year,week,sales,units
0,In_15_Mkts,2017,26,109507.94,51799.0
1,In_15_Mkts,2017,27,95697.34,43435.0
2,In_15_Mkts,2017,28,78909.86,35951.0
3,In_15_Mkts,2017,29,82344.20,38419.0
4,In_15_Mkts,2017,30,79179.16,36439.0
5,In_15_Mkts,2017,31,82270.06,37648.0
6,In_15_Mkts,2018,26,87022.58,46401.0
7,In_15_Mkts,2018,27,80481.82,35431.0
8,In_15_Mkts,2018,28,71876.92,29451.0
9,In_15_Mkts,2018,29,66317.91,26302.0


# 64, 128 respectively comparison

In [17]:
Krogger_both_year_64=Krogger_both_year[Krogger_both_year['product']=="Sunny D 64 FO"]
Krogger_both_year_existing_64=Krogger_both_year_64.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_64=Krogger_both_year_existing_64[Krogger_both_year_existing_64['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year_64=Krogger_both_year_64[Krogger_both_year_64['store_info'].isin(Krogger_both_year_existing_64)]
Krogger_both_year_compare_64=Krogger_both_year_64[Krogger_both_year_64['week'].isin(week_list)]
Krogger_both_year_compare_64=pd.merge(Krogger_both_year_compare_64,Zip_DMA_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_64['15_Mkt_Ind']=Krogger_both_year_compare_64['DMA'].apply(lambda x: In_18(x))
df_YoY_compare_summary_64=Krogger_both_year_compare_64.groupby(['15_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [18]:
Krogger_both_year_128=Krogger_both_year[Krogger_both_year['product']=="Sunny D 128 FO"]
Krogger_both_year_existing_128=Krogger_both_year_128.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_128=Krogger_both_year_existing_128[Krogger_both_year_existing_128['year_week']==12]['store_info'].unique().tolist()
Krogger_both_year_128=Krogger_both_year_128[Krogger_both_year_128['store_info'].isin(Krogger_both_year_existing_128)]
Krogger_both_year_compare_128=Krogger_both_year_128[Krogger_both_year_128['week'].isin(week_list)]
Krogger_both_year_compare_128=pd.merge(Krogger_both_year_compare_128,Zip_DMA_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_128['15_Mkt_Ind']=Krogger_both_year_compare_128['DMA'].apply(lambda x: In_18(x))
df_YoY_compare_summary_128=Krogger_both_year_compare_128.groupby(['15_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [19]:
Krogger_both_year_compare_2_product=Krogger_both_year[(Krogger_both_year['store_info'].isin(Krogger_both_year_existing_64)) &\
                                                     (Krogger_both_year['store_info'].isin(Krogger_both_year_existing_128))]
Krogger_both_year_compare_2_product=Krogger_both_year_compare_2_product[Krogger_both_year_compare_2_product['week'].isin(week_list)]
Krogger_both_year_compare_2_product=pd.merge(Krogger_both_year_compare_2_product,Zip_DMA_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_2_product['15_Mkt_Ind']=Krogger_both_year_compare_2_product['DMA'].apply(lambda x: In_18(x))
df_YoY_compare_summary_2_product=Krogger_both_year_compare_2_product.groupby(['15_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [20]:
Krogger_both_year_compare_2_product.shape

(46586, 12)

In [21]:
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_DMA_YoY_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [22]:
def YoY_summary_df(df):
    df_2017_sales=df[df['year']=='2017'].rename(columns={"sales":"sales_2017"})
    df_2018_sales=df[df['year']=='2018'].rename(columns={"sales":"sales_2018"})
    del df_2017_sales['units']
    del df_2018_sales['units']
    del df_2017_sales['year']
    del df_2018_sales['year']
    sales_yoy=pd.merge(df_2017_sales,df_2018_sales,on=["15_Mkt_Ind",'week'],how="left")
    
    df_2017_units=df[df['year']=='2017'].rename(columns={"units":"units_2017"})
    df_2018_units=df[df['year']=='2018'].rename(columns={"units":"units_2018"})
    del df_2017_units['sales']
    del df_2018_units['sales']
    del df_2017_units['year']
    del df_2018_units['year']
    units_yoy=pd.merge(df_2017_units,df_2018_units,on=["15_Mkt_Ind",'week'],how="left")
    
    output=pd.merge(sales_yoy,units_yoy,on=["15_Mkt_Ind",'week'],how="left")
    output['sales_yoy']=(output['sales_2018']-output['sales_2017'])/output['sales_2017']
    output['units_yoy']=(output['units_2018']-output['units_2017'])/output['units_2017']
    return output

In [23]:
df_YoY_compare_summary_128

,15_Mkt_Ind,year,week,sales,units
0,In_15_Mkts,2017,26,77131.90,33015.0
1,In_15_Mkts,2017,27,63989.54,24873.0
2,In_15_Mkts,2017,28,48541.11,17671.0
3,In_15_Mkts,2017,29,51715.13,20415.0
4,In_15_Mkts,2017,30,51386.49,20684.0
5,In_15_Mkts,2017,31,52760.37,21043.0
6,In_15_Mkts,2018,26,46370.30,14685.0
7,In_15_Mkts,2018,27,52219.16,17194.0
8,In_15_Mkts,2018,28,48461.34,16386.0
9,In_15_Mkts,2018,29,45095.44,14698.0


In [24]:
summary_64=YoY_summary_df(df_YoY_compare_summary_64)
summary_128=YoY_summary_df(df_YoY_compare_summary_128)
summary_both_respective=YoY_summary_df(df_YoY_compare_summary_2_product)
summary_both_together=YoY_summary_df(df_YoY_compare_summary)

In [25]:
summary_64.to_excel(writer,"summary_64",index=False)
summary_128.to_excel(writer,"summary_128",index=False)
summary_both_respective.to_excel(writer,"summary_both_respective",index=False)
summary_both_together.to_excel(writer,"summary_both_together",index=False)

In [26]:
writer.save()

In [27]:
summary_both_respective

,15_Mkt_Ind,week,sales_2017,sales_2018,units_2017,units_2018,sales_yoy,units_yoy
0,In_15_Mkts,26,103650.97,82522.95,49062.0,44261.0,-0.203838,-0.097856
1,In_15_Mkts,27,90510.65,76230.94,40925.0,33525.0,-0.157768,-0.180819
2,In_15_Mkts,28,74383.31,67824.67,33643.0,27625.0,-0.088174,-0.178878
3,In_15_Mkts,29,77572.62,62776.22,36121.0,24785.0,-0.190743,-0.313834
4,In_15_Mkts,30,75146.69,58108.71,34517.0,22336.0,-0.226730,-0.352899
5,In_15_Mkts,31,77846.22,60350.40,35622.0,23665.0,-0.224748,-0.335663
6,Other,26,109446.26,90140.15,51117.0,38601.0,-0.176398,-0.244850
7,Other,27,101447.11,93189.81,45629.0,37077.0,-0.081395,-0.187425
8,Other,28,103244.43,89273.22,53087.0,34836.0,-0.135322,-0.343794
9,Other,29,104082.66,80739.76,57058.0,30785.0,-0.224273,-0.460461
